<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso3/ciclo4/1_cli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1o4udU5qVMi_7jDi0XzSspbPC6Hw0ev9o" width="100%">

# **Introducción a Despliegue de Modelos**
---

En este notebook veremos una introducción al despliegue de modelos con una aplicación tipo CLI desde _Python_.

Comenzamos configurando el servidor de `mlflow` e importando las librerías necesarias:

In [ ]:
!pip install mlflow

In [ ]:
import mlflow
import os
import pandas as pd
from IPython.display import display

Adicionalmente, utilizaremos un servidor de `mlflow`:

In [ ]:
command = """
mlflow server \
        --backend-store-uri sqlite:///tracking.db \
        --default-artifact-root file:mlruns \
        -p 5000 &
"""
get_ipython().system_raw(command)

Utilizaremos `ngrok` para acceder al tablero de `mlflow`:

In [ ]:
!pip install pyngrok

Ahora debe agregar su token de `ngrok`:

In [ ]:
token = "" # Agregue el token dentro de las comillas
os.environ["NGROK_TOKEN"] = token

Nos autenticamos en ngrok:

In [ ]:
!ngrok authtoken $NGROK_TOKEN

Ahora, lanzamos la conexión con ngrok:

In [ ]:
from pyngrok import ngrok
ngrok.connect(5000, "http")

Especificamos que MLFlow debe usar el servidor que estamos manejando.

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")

Vamos a crear un experimento en MLFlow para este conjunto de datos:

In [ ]:
exp_id = mlflow.create_experiment(name="fake_job", artifact_location="mlruns/")

## **1. Carga de Datos**
---

En este caso utilizaremos el conjunto de datos [Real / Fake Job Posting Prediction](https://www.kaggle.com/datasets/shivamb/real-or-fake-fake-jobposting-prediction). Se trata de un conjunto de datos público que contiene información sobre ofertas de trabajo publicadas en línea. El objetivo es predecir si una oferta de trabajo es legítima o falsa. El conjunto de datos está etiquetado con valores binarios que indican si una oferta de trabajo es legítima o falsa.

<center><img src="https://drive.google.com/uc?export=view&id=1I_u3x5jYImlRriBaXyrv_F1snzMFLSbZ" width="80%"></center>

El conjunto de datos contiene 18,000 ofertas de trabajo, de las cuales el 80% son ofertas de trabajo legítimas y el 20% son falsas. Las ofertas de trabajo se recopilaron de diversos sitios web de ofertas de trabajo y se seleccionaron manualmente para garantizar que fueran ofertas de trabajo reales o falsas.

Cada registro en el conjunto de datos contiene información como el título del trabajo, la ubicación, la descripción del trabajo y la empresa que publicó la oferta de trabajo. Además, hay otras características relacionadas con la compañía que publicó la oferta de trabajo, como el tamaño de la compañía, la industria y el tipo de empleador.

Este conjunto de datos es útil para tareas de clasificación binaria y es utilizado comúnmente en la industria y la academia para entrenar modelos de aprendizaje automático que pueden ayudar a identificar automáticamente las ofertas de trabajo falsas en línea.

Vamos a cargarlo:

In [ ]:
data = pd.read_parquet(
        "https://raw.githubusercontent.com/mindlab-unal/mlds6-datasets/main/u4/fake_jobs.parquet"
        ).dropna()

En este caso tenemos dos columnas:

- `description`: texto con la descripción del trabajo.
- `fraudulent`: etiqueta que indica si un trabajo en fraudulento o no.

In [ ]:
display(data.columns)

## **2. Modelamiento**
---

En este caso, utilizaremos conteos de palabras como forma de representar las descripciones de los trabajos por medio de la clase `Tfidfvectorizer`:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

También utilizaremos el modelo de bosques aleatorios para la clasificación:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

En este caso utilizaremos un `Pipeline` para unificar la extracción de características y la clasificación:

In [ ]:
from sklearn.pipeline import Pipeline

Usamos `train_test_split` para particionar el conjunto de datos y evaluar la generalización:

In [ ]:
from sklearn.model_selection import train_test_split
corpus_train, corpus_test, labels_train, labels_test = train_test_split(
    data.description,
    data.fraudulent,
    stratify=data.fraudulent,
    random_state=0,
    test_size=0.3
    )

Ahora, veamos el entrenamiento del modelo:

In [ ]:
with mlflow.start_run(
        run_name="random_forest", experiment_id=exp_id
        ):
    model = Pipeline([
        ("extractor", TfidfVectorizer(max_features=5000, sublinear_tf=True)),
        ("clf", RandomForestClassifier(max_depth=5, n_estimators=200, random_state=0))
        ])
    model.fit(corpus_train, labels_train)
    mlflow.sklearn.log_model(model, "model")
    mlflow.log_metric("accuracy", model.score(corpus_test, labels_test))

Este modelo debería de obtener un desempeño a nivel de `accuracy` de alrededor de `0.96`. Ahora, es importante que genere una versión de este modelo con el nombre **jobclf**.

Con esto, podremos generar predicciones de la siguiente forma:

In [ ]:
model_name = 'jobclf'
model_version = 1
model = mlflow.pyfunc.load_model(f"models:/{model_name}/{model_version}")
display(model)

Podemos usarlo para generar predicciones, para ello, comenzamos definiendo un texto a clasificar:

In [ ]:
text = "Organised - Focused - Vibrant - Awesome!Do you have a passion for customer service? Slick typing skills? Maybe Account Management? ...And think administration is cooler than a polar bear on a jetski? Then we need to hear you!\xa0We are the Cloud Video Production Service and opperating on a glodal level. Yeah, it's pretty cool. Serious about\xa0delivering a world class product and excellent customer service.Our rapidly expanding business is looking for a talented Project Manager to manage the successful delivery of video projects, manage client communications and drive the production process. Work with some of the coolest brands on the planet and learn from a global team that are representing NZ is a huge way!We are entering the next growth stage of our business and growing quickly internationally. \xa0Therefore, the position is bursting with opportunity for the right person entering the business at the right time.\xa090 Seconds, the worlds Cloud Video Production Service -\xa0http://90#URL_fbe6559afac620a3cd2c22281f7b8d0eef56a73e3d9a311e2f1ca13d081dd630#90 Seconds is the worlds Cloud Video Production Service enabling brands and agencies to get high quality online video content shot and produced anywhere in the world. Fast, affordable, and all managed seamlessly in the cloud from purchase to publish.\xa090 Seconds removes the hassle, cost, risk and speed issues of working with regular video production companies by managing every aspect of video projects in a beautiful online experience. \xa0With a growing network of over 2,000 rated video professionals in over 50 countries and dedicated production success teams in 5 countries guaranteeing video project success 100%. It's as easy as commissioning a quick google adwords campaign.90 Seconds has produced almost 4,000 videos in over 30 Countries for over 500 Global brands including some of the worlds largest including Paypal, L'oreal, Sony and Barclays and has offices in Auckland, London, Sydney, Tokyo &amp; Singapore.Our Auckland office is based\xa0right in the heart of the Wynyard Quarter Innovation Precinct - GridAKL!\xa0"

Generamos la predicción:

In [ ]:
prediction = model.predict([text])
print(prediction)

Como podemos ver, este texto no corresponde a una descripción de trabajo fraudulenta.

## **3. Despliegue de Modelos**
---

El despliegue de modelos de machine learning (ML) es el proceso de integrar modelos de aprendizaje automático en un entorno de producción para que puedan utilizarse para hacer predicciones y tomar decisiones.

<center><img src="https://drive.google.com/uc?export=view&id=1IT0IiJaAl6Yallj-Dmt2nLROCd_b9uMa" width="80%"></center>

En el proceso de despliegue de modelos de ML es importante considerar aspectos como la privacidad de los datos, la seguridad, la escalabilidad y el rendimiento para garantizar que el modelo sea confiable y útil en el entorno de producción.

Existen distintos tipos de despliegue de modelos, entre ellos encontramos:

- **CLI**: interfaz de usuario en la que un usuario interactúa con un programa mediante la introducción de comandos de texto simples en una terminal o consola.
- **Aplicaciones de escritorio**: involucran una interfaz gráfica de usuario y permiten interactuar con los modelos por medio de ventanas, botones, cuadros de texto y demás.
- **APIs**: es posible crear APIs para los modelos, tanto librerías de _Python_ reutilizables como aplicaciones web para predicción.

En este notebook veremos un ejemplo con un **CLI** creado en _Python_

## **4. Command Line Interface**
---

**CLI** son las siglas en inglés de *Command Line Interface* (Interfaz de línea de comandos en español). Un CLI se utiliza a menudo en sistemas operativos basados en Unix, como Linux y macOS, así como en algunos sistemas operativos de Microsoft Windows. Los usuarios pueden utilizar comandos específicos para realizar tareas en el sistema, como crear archivos, cambiar permisos de archivo, copiar o mover archivos, y configurar el sistema.

In [ ]:
#@markdown ##**Ejecute esta celda para ver el video.**
from IPython.display import IFrame
IFrame(
        src="https://drive.google.com/file/d/1JBYfu2lE6rqtmQNgZ4q14hBn8-QLc3Sx/preview",
        width="768px",
        height="432px"
        )

El CLI también se utiliza a menudo en programación, especialmente para herramientas de línea de comandos y scripts. Algunos lenguajes de programación tienen bibliotecas que permiten a los desarrolladores crear aplicaciones de línea de comandos para interactuar con los usuarios mediante la entrada de comandos de texto en una terminal.

Este tipo de aplicaciones son muy comunes para el despliegue de modelos de machine learning, en especial es un proceso sencillo que consiste en la implementación de un script que permita recibir información a partir de comandos y retornar predicciones en forma de texto:

<center><img src="https://drive.google.com/uc?export=view&id=1WHvc30OJR6ItI2NhEWN_0QzZ-GE5GNeq" width="80%"><center>

Veamos cómo podemos crear un CLI para el modelo, para esto, debemos crear un script de _Python_ que nos permita recibir textos y retornar predicciones.

Primero creamos la carpeta `src`

In [ ]:
!mkdir src

Definimos el script

In [ ]:
%%writefile ./src/mlcli.py
from argparse import ArgumentParser
import mlflow
mlflow.set_tracking_uri("http://localhost:5000")

def main():
    parser = ArgumentParser(
            description="CLI para modelo de detección de trabajos fraudulentos"
            )
    parser.add_argument("--text", type=str, required=True, help="Texto con la descripción del trabajo")
    args = parser.parse_args()
    model = mlflow.pyfunc.load_model("models:/jobclf/1")
    prediction = model.predict([args.text])[0]
    prediction = "fraudulento" if prediction else "no fraudulento"
    print(f"El trabajo es: {prediction}")

if __name__ == "__main__":
    main()

En este script, utilizamos la librería `argparse`. Esta nos permite estructurar la información recibida por el script y mostrar ayudas en caso de ser necesario:

In [ ]:
!python ./src/mlcli.py --help

Podemos probar el script:

In [ ]:
!python ./src/mlcli.py --text "Organised - Focused - Vibrant - Awesome!Do you have a passion for customer service? Slick typing skills? Maybe Account Management? ...And think administration is cooler than a polar bear on a jetski? Then we need to hear you!\xa0We are the Cloud Video Production Service and opperating on a glodal level. Yeah, it's pretty cool. Serious about\xa0delivering a world class product and excellent customer service.Our rapidly expanding business is looking for a talented Project Manager to manage the successful delivery of video projects, manage client communications and drive the production process. Work with some of the coolest brands on the planet and learn from a global team that are representing NZ is a huge way!We are entering the next growth stage of our business and growing quickly internationally. \xa0Therefore, the position is bursting with opportunity for the right person entering the business at the right time.\xa090 Seconds, the worlds Cloud Video Production Service -\xa0http://90#URL_fbe6559afac620a3cd2c22281f7b8d0eef56a73e3d9a311e2f1ca13d081dd630#90 Seconds is the worlds Cloud Video Production Service enabling brands and agencies to get high quality online video content shot and produced anywhere in the world. Fast, affordable, and all managed seamlessly in the cloud from purchase to publish.\xa090 Seconds removes the hassle, cost, risk and speed issues of working with regular video production companies by managing every aspect of video projects in a beautiful online experience. \xa0With a growing network of over 2,000 rated video professionals in over 50 countries and dedicated production success teams in 5 countries guaranteeing video project success 100%. It's as easy as commissioning a quick google adwords campaign.90 Seconds has produced almost 4,000 videos in over 30 Countries for over 500 Global brands including some of the worlds largest including Paypal, L'oreal, Sony and Barclays and has offices in Auckland, London, Sydney, Tokyo &amp; Singapore.Our Auckland office is based\xa0right in the heart of the Wynyard Quarter Innovation Precinct - GridAKL!\xa0"

Ahora, podemos crear el CLI al crear un archivo `pyproject.toml` el cual contiene las dependencias del proyecto y permite instalar el script dentro del sistema. Veamos el archivo:

In [ ]:
%%writefile ./src/pyproject.toml
[project]
name = "mlcli" # nombre del proyecto
version = "0.1.0" # versión del proyecto
description = "CLI para modelo de detección de trabajos fraudulentos" # descripción
authors = [
    {name="Juan Lara", email="julara@unal.edu.co"},
    ] # autores con correo.

dependencies = [
    "scikit-learn >= 1.2.1", # depende de sklearn
    "mlflow >= 2.1.1" # depende de mlflow
    ]

[build-system]
requires = [
        "setuptools", "wheel"
        ]

[project.scripts]
mlcli = "mlcli:main" # cli

En este caso creamos un CLI con nombre `mlcli` que utiliza la función `main` del script `mlcli.py`.

Instalamos el proyecto:

In [ ]:
!pip install ./src/

Podemos validar que el cli fue creado:

In [ ]:
!mlcli --text "Organised - Focused - Vibrant - Awesome!Do you have a passion for customer service? Slick typing skills? Maybe Account Management? ...And think administration is cooler than a polar bear on a jetski? Then we need to hear you!\xa0We are the Cloud Video Production Service and opperating on a glodal level. Yeah, it's pretty cool. Serious about\xa0delivering a world class product and excellent customer service.Our rapidly expanding business is looking for a talented Project Manager to manage the successful delivery of video projects, manage client communications and drive the production process. Work with some of the coolest brands on the planet and learn from a global team that are representing NZ is a huge way!We are entering the next growth stage of our business and growing quickly internationally. \xa0Therefore, the position is bursting with opportunity for the right person entering the business at the right time.\xa090 Seconds, the worlds Cloud Video Production Service -\xa0http://90#URL_fbe6559afac620a3cd2c22281f7b8d0eef56a73e3d9a311e2f1ca13d081dd630#90 Seconds is the worlds Cloud Video Production Service enabling brands and agencies to get high quality online video content shot and produced anywhere in the world. Fast, affordable, and all managed seamlessly in the cloud from purchase to publish.\xa090 Seconds removes the hassle, cost, risk and speed issues of working with regular video production companies by managing every aspect of video projects in a beautiful online experience. \xa0With a growing network of over 2,000 rated video professionals in over 50 countries and dedicated production success teams in 5 countries guaranteeing video project success 100%. It's as easy as commissioning a quick google adwords campaign.90 Seconds has produced almost 4,000 videos in over 30 Countries for over 500 Global brands including some of the worlds largest including Paypal, L'oreal, Sony and Barclays and has offices in Auckland, London, Sydney, Tokyo &amp; Singapore.Our Auckland office is based\xa0right in the heart of the Wynyard Quarter Innovation Precinct - GridAKL!\xa0"

Como podemos ver, creamos el cli `mlcli` que se puede instalar con `pip`. Esta es una de las formas más simples de despliegue de modelos.

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

- [The pyproject.toml file](https://python-poetry.org/docs/pyproject/).
- [Build a quick inference CLI for your machine learning model](https://medium.com/@sthacruz/build-a-quick-inference-cli-for-your-machine-learning-model-be649cf675be).

## Créditos
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Asistente docente**:

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Coordinador de virtualización:**

- [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Diseño de imágenes:**
  - [Rosa Alejandra Superlano Esquibel](https://www.linkedin.com/in/alejandra-superlano-02b74313a/).
  - [Mario Andrés Rodríguez Triana](mailto:mrodrigueztr@unal.edu.co).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*